In [93]:
import numpy as np
import pandas as pd

In [94]:


# Load the MNIST dataset
mnist_data = np.load('mnist.npz')

# Extract training data and labels
X_train = mnist_data['x_train']
y_train = mnist_data['y_train']
x_test = mnist_data['x_test']
y_test = mnist_data['y_test']

# Storing the indexes
indices = (y_train == 0) | (y_train == 1) | (y_train == 2)

X = X_train[indices]
y_train = y_train[indices]

indices1 = (y_test == 0) | (y_test == 1) | (y_test == 2)

x_test = x_test[indices1]
y_test = y_test[indices1]

# Reshape and flatening the data to 2D
X = X.reshape(X.shape[0], -1)

x_test= x_test.reshape(x_test.shape[0], -1)
# print(X_012.shape)

# Compute mean of the dataset
mean = np.mean(X, axis=0)
mean1 = np.mean(x_test,axis=0)

X = X - mean
x_test=x_test-mean1

# Compute covariance matrix
# print(X_012.T.shape)
cov_matrix = np.cov(X, rowvar=False)

cov_matrix1=np.cov(x_test,rowvar=False)

# cov_matrix = np.cov(X_012.T)
# print(cov_matrix.shape)

# Computing eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eig(cov_matrix1)


# Sorting eigenvectors based on eigenvalues
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvectors_sorted = eigenvectors[:, sorted_indices]

sorted_indices1 = np.argsort(eigenvalues1)[::-1]
eigenvectors_sorted1 = eigenvectors1[:, sorted_indices1]


# Select the top p eigenvectors
p = 10
pca_matrix = eigenvectors_sorted[:, :p]
print(pca_matrix.shape)


pca_matrix1 = eigenvectors_sorted1[:, :p]
print(pca_matrix1.shape)



# Project the data on reduced dimensional space
X_reduced = np.dot(X, pca_matrix)

X_reduced1 = np.dot(x_test, pca_matrix1)


print(X_reduced.shape)
print(X_reduced[0])
print(y_train.shape)
print("Dimensions:", X_reduced.shape)
print(X_reduced1.shape)



(784, 10)
(784, 10)
(18623, 10)
[-1139.38939686+0.j  -587.9349331 +0.j  -584.30307529+0.j
   208.50597448+0.j  -335.67509036+0.j   138.61672253+0.j
   244.73422311+0.j  -111.01412832+0.j   -52.94452131+0.j
     8.25235943+0.j]
(18623,)
Dimensions: (18623, 10)
(3147, 10)


In [95]:
class Node():
    # Constructor
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.value = value
        self.threshold = threshold
        self.feature_index = feature_index
        
        self.left = left
        self.right = right
        self.info_gain = info_gain
        

In [96]:
class DecisionTreeClassifier():
    # Constructor
    def __init__(self, min_samples_split=2, max_depth=2):
        self.max_depth = max_depth
        self.root = None
        self.min_samples_split = min_samples_split
        
    # Function to build a tree
    def build_tree(self, dataset, curr_depth=0):    
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # Conditions
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # fOR checking information is positive or not
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
         
        # Storing best split
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = []
        dataset_right = []
        
        for row in dataset:
            if row[feature_index] <= threshold:
                dataset_left.append(row)
            else:
                dataset_right.append(row)
        
        dataset_left = np.array(dataset_left)
        dataset_right = np.array(dataset_right)
        
        return dataset_left, dataset_right

    
    #Finding information gain and then comparing
    def information_gain(self, parent, l_child, r_child):        
        weight_r = len(r_child) / len(parent)
        weight_l = len(l_child) / len(parent)
        gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        return gain
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    #Visualizing the tree
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

# Fitting model

In [97]:


print(X_reduced.shape)
y_train=y_train.reshape(-1,1)
print(y_train.shape)

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
dataset = np.concatenate((X_reduced[:1000],y_train[:1000]), axis=1)
classifier.root = classifier.build_tree(dataset)
# classifier.fit()
classifier.print_tree()

(18623, 10)
(18623, 1)
X_0 <= (624.6639084076752+0j) ? 0.2983877586057319
 left:X_1 <= (179.67561986817168+0j) ? 0.3037225165600382
  left:X_0 <= (-202.45652113485986+0j) ? 0.14759057627204
    left:0j
    right:(2+0j)
  right:X_4 <= (-720.9898876115241+0j) ? 0.0891037058316576
    left:0j
    right:(2+0j)
 right:X_2 <= (231.62107754431915+0j) ? 0.06740047510098737
  left:X_7 <= (-575.7336812339543+0j) ? 0.0056004210280101545
    left:(2+0j)
    right:(1+0j)
  right:X_1 <= (-480.64812928740986+0j) ? 0.2659279778393353
    left:(1+0j)
    right:(2+0j)


# Testing model

In [98]:
# print(np.real(X_reduced1[:10]))
# print(y_test)

Y_pred = [classifier.make_prediction(x, classifier.root) for x in np.real(X_reduced1)]
# Y_pred = classifier.predict(np.real(X_reduced1)) 
# print(Y_pred)
# print(X_reduced1.shape)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, np.real(Y_pred))
# Y_pred = np.array(Y_pred)


0.6619002224340642

In [99]:
predictions, labels=np.array(Y_pred), np.array(y_test)
unique_classes = np.unique(labels)
class_wise_accuracy = {}
for class_val in unique_classes:
    class_indices = np.where(labels == class_val)[0]
    class_accuracy = np.sum(predictions[class_indices] == labels[class_indices]) / len(class_indices)
    class_wise_accuracy[class_val] = class_accuracy



# class_wise_accuracy = calculate_class_wise_accuracy()


for class_val, accuracy in class_wise_accuracy.items():
    print(f"Accuracy for class {class_val}: {accuracy}")

Accuracy for class 0: 0.32857142857142857
Accuracy for class 1: 0.9145374449339208
Accuracy for class 2: 0.7005813953488372


In [100]:
from collections import Counter

def majority_voting(trees, X_test):
    predictions = []
    for tree in trees:
        prediction = []
        for x in X_test:
            pred = tree.make_prediction(x, tree.root)
            prediction.append(pred)
        predictions.append(prediction)
    majority_votes = np.array(predictions)
    majority_votes = np.swapaxes(majority_votes, 0, 1)
    final_predictions = []
    print(majority_votes)
    for votes in majority_votes:
        
        vote_count = Counter(votes)
        final_predictions.append(vote_count.most_common(1)[0][0])
    return final_predictions


X_train, y_train = X_reduced[:1000], y_train[:1000]
datasets = []
for _ in range(5):
    indices = np.random.choice(len(X_train), 100, replace=True)
    X_sample,y_sample = X_train[indices], y_train[indices]
    datasets.append((X_sample, y_sample))


trees = []
for X_sample, y_sample in datasets:
    tree = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
    dataset = np.concatenate((X_sample, y_sample), axis=1)
    tree.root = tree.build_tree(dataset)
    trees.append(tree)

predictions = majority_voting(trees, X_reduced1)

total_accuracy = np.mean(predictions == y_test)

class_accuracy = {}
for class_label in np.unique(y_test):
    correct = np.sum((predictions == y_test) & (y_test == class_label))
    total = np.sum(y_test == class_label)
    class_accuracy[class_label] = correct / total

print("Total Accuracy:", total_accuracy)
print("Class-wise Accuracy:", class_accuracy)


[[0.+0.j 2.+0.j 2.+0.j 2.+0.j 2.+0.j]
 [1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j 2.+0.j]
 ...
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j 2.+0.j]
 [1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
Total Accuracy: 0.6139180171591992
Class-wise Accuracy: {0: 0.6571428571428571, 1: 0.9409691629955947, 2: 0.2131782945736434}
